## Non valid neighborhood removing

### Load data

In [24]:
from pathlib import Path
import geopandas as gpd

def load_data(nombre_archivo, directorio_base='../../data/input'):
    """
    Carga un archivo GeoJSON específico y devuelve un GeoDataFrame.
    
    Args:
        nombre_archivo (str): Nombre del archivo GeoJSON (ej. 'barris.geojson').
        directorio_base (str): Ruta relativa al directorio donde se encuentra el archivo.
                              Por defecto: '../../data/delivetter'.
    
    Returns:
        gpd.GeoDataFrame: GeoDataFrame con los datos del archivo.
    """
    try:
        ruta = Path(directorio_base) / nombre_archivo
        
        if not ruta.exists():
            raise FileNotFoundError(f"File not found: {ruta.absolute()}")
        
        gdf = gpd.read_file(ruta)
        print(f"✅ '{nombre_archivo}' successfully loaded ({len(gdf)} records)")
        return gdf
    
    except Exception as e:
        print(f"❌ Error loading '{nombre_archivo}': {str(e)}")
    
barrios = load_data('barris.geojson')
catastro = load_data('catastro.geojson')
puntos_carga = load_data('puntosCID.geojson')

✅ 'barris.geojson' successfully loaded (88 records)
✅ 'catastro.geojson' successfully loaded (36346 records)
✅ 'puntosCID.geojson' successfully loaded (882 records)


### Filter

In [25]:
nombres = set(barrios['nombre'])

for i, row in barrios.iterrows():
    shp_zone = row.geometry
    shp_loading_points = puntos_carga[puntos_carga.intersects(shp_zone)]
    shp_bajos = catastro[catastro.intersects(shp_zone)]

    cid = len(shp_loading_points)
    almacen = len(shp_bajos[shp_bajos['Almacen'] == True])
    comercio = len(shp_bajos[shp_bajos['Comercio'] == True])

    if cid == 0:
        nombres.remove(row['nombre'])
        print(f"Zona {row['nombre']}: No hay puntos de carga")

    elif almacen < 3:
        nombres.remove(row['nombre'])
        print(f"Zona {row['nombre']}: No hay suficientes almacenes")

    elif comercio < 20:
        nombres.remove(row['nombre'])
        print(f"Zona {row['nombre']}: No hay suficientes comercios")

print(f"\n\nZonas válidas:")
for nombre in nombres:
    print(f" - {nombre}")

Zona EL PALMAR: No hay suficientes comercios
Zona EL PERELLONET: No hay suficientes comercios
Zona SOTERNES: No hay suficientes almacenes
Zona LA FONTSANTA: No hay suficientes almacenes
Zona BORBOTO: No hay puntos de carga
Zona BENIFARAIG: No hay suficientes comercios
Zona EL PILAR: No hay puntos de carga
Zona POBLE NOU: No hay puntos de carga
Zona RAFALELL-VISTABELLA: No hay puntos de carga
Zona BENIFERRI: No hay suficientes comercios
Zona EL SALER: No hay puntos de carga
Zona CARPESA: No hay puntos de carga
Zona EL CARME: No hay puntos de carga
Zona TRES FORQUES: No hay suficientes almacenes
Zona LES CASES DE BARCENA: No hay puntos de carga
Zona MAHUELLA-TAULADELLA: No hay puntos de carga
Zona MASSARROJOS: No hay suficientes comercios
Zona LA FONTETA S.LLUIS: No hay suficientes comercios
Zona LA PUNTA: No hay suficientes comercios
Zona LA LLUM: No hay suficientes comercios
Zona CIUTAT UNIVERSITARIA: No hay suficientes comercios
Zona FAITANAR: No hay puntos de carga
Zona EL FORN D'ALC

### Remove

In [26]:
barrios_filtrados = barrios[barrios['nombre'].isin(nombres)]
barrios_filtrados = barrios_filtrados.reset_index(drop=True)
barrios_filtrados.to_file('../../data/input/barris.geojson', driver="GeoJSON")
print(f"✅  Barrios disponibles: {len(barrios_filtrados)}")


✅  Barrios disponibles: 63
